下面的程序示例大部分来自 https://tensorflow.google.cn/guide/datasets

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.enable_eager_execution()

# 创建Dataset对象

## 从csv文件创建Dataset

## 从数组（Tensor）创建Dataset

In [2]:
data = tf.range(9) # or np.range(9)
print(data)
ds = tf.data.Dataset.from_tensor_slices(data)
print(ds.output_shapes,ds.output_types)

tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
() <dtype: 'int32'>


## 按照第一维切分多维数组（Tensor）
下面的例子将一个2*3的二维数组切分为2个shape为(3,)的向量，也就是说，from_tensor_slices总是将给定的张量降低一个维度：去掉了第一维，output_shapes表达了dataset中每一个元素的维度。

Q: 如何遍历Dataset？

In [28]:
data = tf.random.uniform([4,10])
print(data),print()
dataset1 = tf.data.Dataset.from_tensor_slices(data)
# 从dataset1取出2行来看看
print(list(dataset1.take(2))),print()
print(dataset1.output_shapes,dataset1.output_types)
data = tf.random.uniform([2,3,4])
ds = tf.data.Dataset.from_tensor_slices(data)
print(ds.output_shapes,ds.output_types)

tf.Tensor(
[[0.6369455  0.75700355 0.9008994  0.05953217 0.23252475 0.9033592
  0.17105424 0.48633683 0.32002068 0.7356299 ]
 [0.61686313 0.8060852  0.1563648  0.32801402 0.7224113  0.8921137
  0.45901847 0.7417455  0.64629114 0.24317098]
 [0.01582301 0.90439    0.01361418 0.44813073 0.49562013 0.5387732
  0.8756645  0.7021924  0.45619965 0.61684966]
 [0.5007888  0.07209098 0.8013929  0.73403394 0.8535296  0.18640411
  0.7101948  0.41234338 0.47507048 0.7388607 ]], shape=(4, 10), dtype=float32)

[<tf.Tensor: id=495, shape=(10,), dtype=float32, numpy=
array([0.6369455 , 0.75700355, 0.9008994 , 0.05953217, 0.23252475,
       0.9033592 , 0.17105424, 0.48633683, 0.32002068, 0.7356299 ],
      dtype=float32)>, <tf.Tensor: id=496, shape=(10,), dtype=float32, numpy=
array([0.61686313, 0.8060852 , 0.1563648 , 0.32801402, 0.7224113 ,
       0.8921137 , 0.45901847, 0.7417455 , 0.64629114, 0.24317098],
      dtype=float32)>]

(10,) <dtype: 'float32'>
(3, 4) <dtype: 'float32'>


要求两个Tensor的第一维的维度相同，but why？

In [7]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"

(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))


In [8]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes)  # ==> "(10, ((), (100,)))"

(tf.float32, (tf.float32, tf.int32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


In [13]:
print(dataset3.take(1))

<DatasetV1Adapter shapes: ((10,), ((), (100,))), types: (tf.float32, (tf.float32, tf.int32))>


# 迭代器
## 单次迭代

In [33]:
dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
for i in range(100):
    assert i == iterator.get_next().numpy()